In [ ]:
from pathlib import Path
import torch
import pandas as pd
import stat
import numbers
import pydicom
import numpy as np
from tqdm import tqdm
from collections import Counter
import re
import os

In [ ]:
Path().resolve()

In [ ]:
nas_loc = Path('/home/buehlern/neocortex-nas')

#data_loc = nas_loc / 'shared/Skelett'
data_loc = nas_loc / 'shared/Skelett/HAND_NEU'

foldercontents_cacheloc = Path('cache-hand/prep_foldercontents.pt')
df_loc = Path('cache-hand/prep_df.pkl')
df_allfiles_loc = Path('cache-hand/prep_df_allfiles')

In [ ]:
earlycols = ['patientid', 'bodypart', 'pixelarr_dtype',
             'pixelarr_shape', 'pixelarr_non0count']

# Step 1: Stat items

In [ ]:
def listitems(d):
    print('rglobbing', d)
    return sorted([x for x in d.rglob('*') if not '/__' in str(x)])

In [ ]:
def listitems_stated(d):
    itemsraw = listitems(d)
    print('stating items in', d)
    res = []
    for curres in itemsraw:
        res.append((curres, curres.stat()))
    return res

In [ ]:
foldercontents = listitems_stated(data_loc)
torch.save(foldercontents, foldercontents_cacheloc)
print("saved to", foldercontents_cacheloc.resolve())

# Step 2: Read raw metadata

In [ ]:
allitems = torch.load(foldercontents_cacheloc)

In [ ]:
df_allfiles = pd.DataFrame([{'path': f, 'pathstr': str(f), 'stat': s} for (f, s) in allitems if stat.S_ISREG(s.st_mode)])
df_allfiles.set_index('pathstr', inplace=True)
df_allfiles['path_withoutsquarebrackets'] = df_allfiles.index.str.replace(r'\[\d+\]$', r'', regex=True)

In [ ]:
def read_metainfos(pathstr, read_fulldcm=True):
    output_dict = {'header': '', 'errors': ''} 
    
    try:
        header = pydicom.dcmread(pathstr, stop_before_pixels=True)
        output_dict = {'header': header, 'errors': ''}
    except Exception as e:
        output_dict['errors'] += f'{repr(e)}\n'
    
    if read_fulldcm:
        try:
            dcm = pydicom.dcmread(pathstr)
            if repr(set(dcm.keys()) - set(header.keys())) != '{(7fe0, 0010)}':
                output_dict['errors'] += f'unusual key difference: set(dcm.keys()) - set(header.keys())\n'
        except Exception as e:
            output_dict['errors'] += f'{repr(e)}\n'
        
        try:
            pixelarr = dcm.pixel_array
            output_dict['pixelarr_dtype'] = pixelarr.dtype
            output_dict['pixelarr_shape'] = pixelarr.shape
            output_dict['pixelarr_min'] = np.min(pixelarr)
            output_dict['pixelarr_max'] = np.max(pixelarr)
            output_dict['pixelarr_mean'] = np.mean(pixelarr)
            output_dict['pixelarr_std'] = np.std(pixelarr)
            output_dict['pixelarr_non0count'] = np.count_nonzero(
                pixelarr)
            pixelarr_non0 = pixelarr[pixelarr != 0]
            output_dict['pixelarr_non0min'] = np.min(pixelarr_non0)
            output_dict['pixelarr_non0mean'] = np.mean(
                pixelarr_non0)
            output_dict['pixelarr_non0std'] = np.std(pixelarr_non0)
        except Exception as e:
            output_dict['errors'] += f'{repr(e)}\n'
    
    if output_dict['errors'] != '':
        print(pathstr, output_dict['errors'])
    
    return pathstr, output_dict

In [ ]:
new_keys = read_metainfos(list(df_allfiles.index)[0])[1].keys()
df_allfiles = df_allfiles.reindex(columns=list(df_allfiles.columns) + list(new_keys))

In [ ]:
# for new_key in new_keys:
#     df_allfiles[new_keys] = [np.nan] * len(df_allfiles)

In [ ]:
for pathstr, output_dict in map(read_metainfos, list(df_allfiles.index)):
    for k, v in output_dict.items():
        if not isinstance(v, numbers.Number):
            # ease into setting the cellvalue to an object
            df_allfiles.at[pathstr, k] = ''
        df_allfiles.at[pathstr, k] = v

In [ ]:
df_allfiles.to_pickle(df_allfiles_loc)
print('wrote df of len', len(df_allfiles))

# Step 3: Cleanup

In [ ]:
print('loading df_allfiles')
df_allfiles = pd.read_pickle(df_allfiles_loc)

In [ ]:
# cleanup tool
print('ensuring no cleanup due')
planned_for_deletion = {}
for i, row in tqdm(list(df_allfiles.iterrows())):
    imagetype = []
    try:
        imagetype = row['header'].ImageType
        if 'DRAWING' in imagetype:
            planned_for_deletion[row['path']] = f'{imagetype=}'
    except AttributeError:
        pass

In [ ]:
if len(planned_for_deletion) > 0:
    for i, row in tqdm(list(df_allfiles.iterrows())):
        if row['path'] not in planned_for_deletion:
            continue
            
        ok = False
        for _, innerrow in df_allfiles.iterrows():
            if (not isinstance(innerrow['header'], str)) and innerrow['header'].StudyInstanceUID == row['header'].StudyInstanceUID and innerrow['header'].PatientID == row['header'].PatientID and  innerrow['path'] not in planned_for_deletion:
                ok = True
                break
        #assert ok
        if not ok:
            print("ASSERTION ok:", ok)


    Path('cleanupscript.sh').write_text(
        '\n'.join(
            f"rm '{k}'  # reason: {v}"
            for k, v in planned_for_deletion.items()
        )
    )
    print('cleanup due, cf cleanupscript.sh ; rerun this script after the cleanup and removing the old cache')

# Step 4: Format and drop rows

In [ ]:
df = df_allfiles

In [ ]:
beforelen = len(df)
df = df[df.index == df['path_withoutsquarebrackets']]
print(
    f'dropped {beforelen-len(df)}/{beforelen}={(beforelen-len(df))/beforelen:.4f} (square bracket duplicates)')

In [ ]:
beforelen = len(df)
df = df.iloc[(np.nonzero(df['pixelarr_shape'].fillna(0).to_numpy()))[0]]
print(
    f'dropped {beforelen-len(df)}/{beforelen}={(beforelen-len(df))/beforelen:.4f} (no pixel data)')

In [ ]:
df['patientid'] = [path.parent.parent.parent.name for path in df['path']]
df['bodypart'] = [path.parent.parent.parent.parent.name for path in df['path']]

In [ ]:
# expand df columns by all dicom header fields
dicom_headers_counter = Counter()

for header in tqdm(list(df['header'])):
    dicom_headers_counter.update(header.keys())

dcmcols = [f'dcm_{pydicom.datadict.keyword_for_tag(k)}' for k in sorted(
    dicom_headers_counter)]

df = df.reset_index()
df = df.reindex(columns=earlycols +
                dcmcols + [x for x in list(df.columns) if x not in earlycols])

# df = df.style.hide('header', axis='columns')

for col in tqdm(dcmcols):
    # print('fetching all ', col)
    tag = pydicom.datadict.tag_for_keyword(col[len('dcm_'):])
    df[col] = [
        header[tag].value if tag in header else np.nan for header in df['header']]

In [ ]:
# FILTER OUT UNUSUAL dcm_ImageType s
beforelen = len(df)
df['dcm_ImageType_str'] = [repr(x) for x in df['dcm_ImageType']]
valcounts = df['dcm_ImageType_str'].value_counts()
print(valcounts)
count_images_except_top2imagetypes = valcounts.sum() - valcounts.nlargest(2).sum()
print('count images, except top 2: ',
      count_images_except_top2imagetypes)
keep_imagetypes = set(valcounts.nlargest(2).index)

df_dropimagetypes = df[~df['dcm_ImageType_str'].isin(keep_imagetypes)]
df = df[df['dcm_ImageType_str'].isin(keep_imagetypes)]

for i, droprow in df_dropimagetypes.iterrows():
    subdf = df[(df['dcm_StudyInstanceUID'] == droprow['dcm_StudyInstanceUID']) & (
        df['dcm_PatientID'] == droprow['dcm_PatientID'])]
    #assert len(subdf) > 0
    if not len(subdf) > 0:
        print("ASSERTION len(subdf) > 0:", len(subdf) > 0)

print(f'dropped {beforelen-len(df)}/{beforelen}={(beforelen-len(df))/beforelen:.4f} (unusual image type, all StudyInstanceUID (and PatientID) s remain present)')

In [ ]:
beforelen = len(df)
df_dropimageshape = df[[x[1] == 650 for x in df['pixelarr_shape']]]
df = df[[x[1] != 650 for x in df['pixelarr_shape']]]
for i, droprow in df_dropimagetypes.iterrows():
    subdf = df[(df['dcm_StudyInstanceUID'] == droprow['dcm_StudyInstanceUID']) & (
        df['dcm_PatientID'] == droprow['dcm_PatientID'])]
    #assert len(subdf) > 0
    if not len(subdf) > 0:
        print("ASSERTION len(subdf) > 0:", len(subdf) > 0)
print(f'dropped {beforelen-len(df)}/{beforelen}={(beforelen-len(df))/beforelen:.4f} (unusual image shape, all StudyInstanceUID (and PatientID) s remain present)')

In [ ]:
beforelen = len(df)
# 2 or 3 dimensions
#assert all([2 <= len(x) <= 3 for x in df['pixelarr_shape']])
print("ASSERTION all([2 <= len(x) <= 3 for x in df['pixelarr_shape']])", all([2 <= len(x) <= 3 for x in df['pixelarr_shape']]))
# rgb if 3 dimensions dimensions
#assert all([x[2] == 3 for x in df['pixelarr_shape'] if len(x) == 3])
print("ASSERTION all([2 <= len(x) <= 3 for x in df['pixelarr_shape']])", all([2 <= len(x) <= 3 for x in df['pixelarr_shape']]))
df = df[[len(x) != 3 for x in df['pixelarr_shape']]]
print(f'dropped {beforelen-len(df)}/{beforelen}={(beforelen-len(df))/beforelen:.4f} (rgb images; all images are 2d now (i.e. no color dimension))')

In [ ]:
df.to_pickle(df_loc)

# Step 5: Further formatting

In [ ]:
print('reading df.pkl')
df_loaded = pd.read_pickle(df_loc)

In [ ]:
dcmcols = [x for x in df_loaded.columns if x.startswith('dcm_')]
pivotmask = earlycols + dcmcols

In [ ]:
df_loaded['dcm_AnatomicRegionSequence_str'] = [','.join((x[(0x8, 0x104)].value for x in seq)
                                                 if not seq != seq else '').lower()
                                        for seq in df_loaded['dcm_AnatomicRegionSequence']]

In [ ]:
df_loaded['dcm_BodyPartExamined_str'] = [(seq if not seq != seq else '').lower()
                                  for seq in df_loaded['dcm_BodyPartExamined']]

df_labelcomparison = pd.pivot_table(df_loaded, index=['dcm_BodyPartExamined_str'], columns=['bodypart'], values=['patientid'],
                                    sort=False, aggfunc=lambda x: x.count())

df_labelcomparison

In [ ]:
vowel_char_map = {ord('ä'): 'ae',
                  ord('ö'): 'oe',
                  ord('ü'): 'ue',
                  ord('Ä'): 'Äe',
                  ord('Ö'): 'Oe',
                  ord('Ü'): 'Ue',
                  ord('ß'): 'ss'}

df_loaded['oldbodypart'] = df_loaded['bodypart']
df_loaded['bodypart'] = [re.sub(r'^DX[-_]', '', x).translate(vowel_char_map).lower()
                        for x in df_loaded['bodypart']]

# Step 6: Findings

In [ ]:
findings_loc = nas_loc / 'shared/Befundexport'
all_findings = sorted(findings_loc.rglob('Befunde_*/*.txt'))

In [ ]:
all_findings = dict(zip([re.fullmatch(r'Befund__?(.+)\.txt', x.name).group(1) for x in all_findings], all_findings))

In [ ]:
keyerrors = []
df['findingspath'] = ''
df['findings'] = ''
df['examinationid'] = [x.parent.parent.name for x in df['path']]
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        findingsfile = all_findings[row['examinationid']]
        df.loc[i, 'findingspath'] = findingsfile

        findings = findingsfile.read_text('utf8')
        assert len(findings.strip()) > 0
        df.loc[i, 'findings'] = findings
    except KeyError:
        keyerrors.append(row)

print('NO FINDINGS FOR ', len(keyerrors), ' OF ', len(df))

In [ ]:
pd.to_pickle(df, df_loc)
print('WROTE FINAL DF (including findings)')